In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input

from mobilenet import get_mobilenet

In [ ]:
data_dir = '/home/ubuntu/data/'

In [ ]:
data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=preprocess_input
)

train_generator = data_generator.flow_from_directory(
    data_dir + 'train', 
    target_size=(224, 224),
    batch_size=64
)

val_generator = data_generator.flow_from_directory(
    data_dir + 'val', 
    target_size=(224, 224),
    batch_size=64
)

In [ ]:
model = get_mobilenet()

for layer in model.layers[:-2]:
    layer.trainable = False
    
model.layers[-2].kernel_regularizer = keras.regularizers.l2(1e-3)

In [ ]:
model.compile(
    optimizer=optimizers.Adam(lr=1e-3), 
    loss='categorical_crossentropy', metrics=['accuracy']
)

In [ ]:
model.fit_generator(
    train_generator, 
    steps_per_epoch=150, epochs=10, 
    validation_data=val_generator, validation_steps=16, 
    max_queue_size=10, workers=1, use_multiprocessing=False
)